## PHYMOL Training School 3, August 2025

---
# Molecular properties
## Example Workflow

### 0. Molecule and basis set

In [ ]:
import veloxchem as vlx

In [ ]:
# ethene molecule
ethene_xyz = """6

C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
"""

molecule = vlx.Molecule.from_xyz_string(ethene_xyz)
basis = vlx.MolecularBasis.read(molecule, "def2-svp")

### 1. SCF

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

### 2. Polarizability

Calculated from the electric dipole-electric dipole linear response function:
$$\alpha_{\alpha\beta}(\omega, -\omega) = - \langle\langle\mu_\alpha,\mu_\beta\rangle\rangle_\omega$$

In [ ]:
lrf_drv = vlx.LinearResponseSolver() 
lrf_drv.a_operator = "electric dipole"
lrf_drv.b_operator = "electric dipole"

lrf_drv.frequencies = [0.0656]

lrf_results = lrf_drv.compute(molecule, basis, scf_results)

In [ ]:
lrf = lrf_results.get('response_functions')
alpha_iso = -1/3 * (lrf['x', 'x', 0.0656] + lrf['y', 'y', 0.0656] + lrf['z', 'z', 0.0656])

print(f"Polarizability isotropic average: {alpha_iso} ")

### 3. $C_6$ coefficients

$$C_6 = \frac{3\hbar}{\pi} \int_0^\infty \bar\alpha_A(i\omega)\bar\alpha_B(i\omega)d\omega$$

In [ ]:
c6_drv = vlx.C6Driver()
scf_results = scf_drv.compute(molecule, basis)
c6_results = c6_drv.compute(molecule, basis, scf_results)

c6 = c6_results["c6"]

### 4. UV-Vis absorpton

Oscillator strengths of electronic transitions
$$f_{0n} = \frac{2}{3}\omega_{0n}\sum_{\alpha=x,y,z}|\langle 0|\mu_\alpha|n\rangle|^2$$

In [ ]:
lreig_drv = vlx.LinearResponseEigenSolver()
lreig_drv.nstates = 4
lreig_results = lreig_drv.compute(molecule, basis, scf_results)

In [ ]:
lreig_drv.plot_uv_vis(lreig_results)